In [ ]:
#import libraries
import numpy 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import metrics
import matplotlib.patches as mpatches
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import scipy.stats as st 
# import dtale 
import warnings
warnings.filterwarnings('ignore')
import pandas_profiling as pdp
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve

In [ ]:
fd_dlvry=pd.read_csv('../input/online-food-delivery-preferencesbangalore-region/onlinedeliverydata.csv')

In [ ]:
fd_ori=fd_dlvry.copy(deep=True)
#future uses one copy create

In [ ]:
fd_dlvry.shape

In [ ]:
fd_dlvry.head()

In [ ]:
fd_dlvry.info()

In [ ]:
#we can use datle library for EDA but 
# d = dtale.show(fd_dlvry)
# d.open_browser()


In [ ]:
# looking data with pdp.ProfileReport 
data = pd.read_csv('../input/online-food-delivery-preferencesbangalore-region/onlinedeliverydata.csv')

#Creating the Exploratory Data Analysis of our dataset
report = pdp.ProfileReport(data, title='Pandas Profiling Report')

In [ ]:
report

In [ ]:
pip install datacleaner

In [ ]:
#this library i found on google it's helpful for cleaning data,imputing value and lable encoding
import  datacleaner as dc # pip install datacleaner

In [ ]:
dc.autoclean(fd_dlvry).head()

In [ ]:
fd_dlvry.head() 

In [ ]:
fd_ori.head()

In [ ]:
l1=(fd_ori.columns[:])
l1

In [ ]:
fd_dlvry.info()

In [ ]:
df=fd_ori.drop('Reviews',axis=1) #drop reviews because it's required  NLP in future i will try it
df.columns

In [ ]:
%matplotlib inline

In [ ]:
# we needs to back map for lable encoding 
lbl=dict()
for i in fd_ori.columns:
    if i not in  ['latitude','longitude','Pin code','Output','Reviews']:
                  lbl[i]=dict(zip(fd_ori[i].unique(), fd_dlvry[i].unique()))
    

In [ ]:
lbl

In [ ]:
# # Optional practice chi-squared test 
# df=pd.crosstab(fd_ori['Meal(P1)'],fd_ori['Occupation'])
# df

# stat, p, dof, expected = st.chi2_contingency(df,correction=True)
# stat, p, dof, expected




<b> method 1: Feature selection Cramér's V </b>

In [ ]:
#Cramer's V perform for find correlation between catgorical variable with respect to Output
crv={}
l1=[]
for i in fd_ori.columns:
    if i not in  ['latitude','longitude','Pin code','Output','Reviews']:
        #we are not using lat and long because in EDA all location diffrent (reference dtale lib)
        df=pd.crosstab(fd_ori['Output'],fd_ori[i])
        stat, p, dof, expected = st.chi2_contingency(df,correction=True)
        t=min(df.shape)-1
        deno=sum(df.sum())*t
        x=numpy.sqrt(stat / deno)
        if(x>0.41): #thresold
             l1.append((i,x))


crv['Output']=l1
        
crv

In [ ]:
#Extract the feature from above dictionary 
basic_f0=[]
for i in l1:
    basic_f0.append(i[0])

basic_f0

In [ ]:
X=fd_dlvry[basic_f0] # select X 
y=fd_dlvry['Output'] # select Y


In [ ]:
basic_f0

In [ ]:
sns.countplot(fd_ori['Output']) # look target feature

<h1> KNN Algorithm </h1>

In [ ]:
# train_test_split by sklearn 
X=fd_dlvry[basic_f0]
y=fd_dlvry['Output']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33)

In [ ]:
#select best knneighbour find 
plt.figure()
training_accuracy = []
test_accuracy = []
# try n_neighbors from 1 to 10
neighbors_settings = range(1, 11)
for i in neighbors_settings:
    clf = KNeighborsClassifier(n_neighbors=i)
    clf.fit(X_train, y_train)
    training_accuracy.append(clf.score(X_train, y_train))
    # record generalization accuracy
    test_accuracy.append(clf.score(X_test, y_test))
plt.plot(neighbors_settings,training_accuracy,label="training accuracy")
plt.plot(neighbors_settings,test_accuracy,label="test accuracy")
plt.ylabel("Accuracy")
plt.xlabel("n_neighbors")
plt.legend()

plt.show()

In [ ]:
# appropriate test size find 
plt.figure()
training_accuracy = []
test_accuracy = []
# try n_neighbors from 1 to 10
test_size= numpy.arange(0.1, 0.9,0.05)
knn = KNeighborsClassifier(n_neighbors = 5,p=1)
for i in test_size:
    
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = i)
        knn.fit(X_train, y_train)
        training_accuracy.append(knn.score(X_test, y_test))
        test_accuracy.append(knn.score(X_train, y_train))
  

    
plt.plot(test_size,training_accuracy,label="training accuracy")
plt.plot(test_size,test_accuracy,label="test accuracy")
plt.ylabel("Accuracy")
plt.xlabel("test_size")
plt.legend()

plt.show()

In [ ]:
#split data
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33)

In [ ]:
# fit data and perform the knn algo
neigh = KNeighborsClassifier(n_neighbors=5,p=1)

knn=neigh.fit(X_train,y_train)

knn.score(X_test,y_test)


In [ ]:
#y_pred for confusion matrix
y_pred=knn.predict(X_test)

In [ ]:
#confusion matrix 
cf_matrix_n=confusion_matrix(y_test, y_pred)
cf_matrix_n

In [ ]:
#classification report 
print(classification_report(y_test,y_pred))

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds=roc_curve(y_test, y_pred)
AUC=round(metrics.auc(fpr, tpr),2)

plt.plot([0,1], [0,1], linestyle='--', label='No Skill')
plt.plot(fpr, tpr, marker='.', label=f'knn AUC={AUC}')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
# show the plot
plt.show()


<b> Conclusion: </b>

In [ ]:
'''Here weighted avg f1-score is good and AUC is greate but problems here Knn is not learning something
knn just use more memory space for perform operation and mostly features are categorical so noise affect
on this algorithms
'''

<b>method 2: feature selection using sklearn </b>

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2,mutual_info_classif

In [ ]:
def select_features(X_train, y_train, X_test):
    fs = SelectKBest(score_func=chi2, k='all')
    fs.fit(X_train, y_train)
    X_train_fs = fs.transform(X_train)
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

In [ ]:
#select feature 
l1=['latitude','longitude','Pin code','Output','Reviews']
X=fd_dlvry[fd_dlvry.columns[~fd_dlvry.columns.isin(l1)]]
y=fd_dlvry['Output']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [ ]:
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test)
for i in range(len(fs.scores_)):
     print('Feature %d: %f' % (i, fs.scores_[i]))
# plot the scores
plt.bar([i for i in range(len(fs.scores_))], fs.scores_)

plt.show()

In [ ]:
# Extract feature 
basic_f1=[]
for i in range(len(fs.scores_)):
    if(fs.scores_[i]>10): # set Thresold
        print(X_test.columns[i])
        basic_f1.append(X_test.columns[i])

In [ ]:
X=fd_dlvry[basic_f1]
y=fd_dlvry['Output']

In [ ]:
#neighbour find 
plt.figure()
training_accuracy = []
test_accuracy = []
# try n_neighbors from 1 to 10
neighbors_settings = range(1, 11)
for i in neighbors_settings:
    clf = KNeighborsClassifier(n_neighbors=i,p=1)
    clf.fit(X_train, y_train)
    training_accuracy.append(clf.score(X_train, y_train))
    # record generalization accuracy
    test_accuracy.append(clf.score(X_test, y_test))
plt.plot(neighbors_settings,training_accuracy,label="training accuracy")
plt.plot(neighbors_settings,test_accuracy,label="test accuracy")
plt.ylabel("Accuracy")
plt.xlabel("n_neighbors")
plt.legend()

plt.show()

In [ ]:
# here some Extra work it's take long time to running
# plt.figure()
# training_accuracy = []
# test_accuracy = []
# # try n_neighbors from 1 to 10
# test_size= numpy.arange(0.1, 0.9,0.05)
# knn = KNeighborsClassifier(n_neighbors = 3)
# for i in test_size:
    
    
#     test_score = []
#     train_score=[]
#     for j in range(1,1000):
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = i,random_state=6)
#         knn.fit(X_train, y_train)
#         test_score.append(knn.score(X_test, y_test))
#         train_score.append(knn.score(X_train, y_train))
  

       
#     training_accuracy.append(np.mean(train_score))
#     # record generalization accuracy
#     test_accuracy.append(np.mean(test_score))

# plt.plot(test_size,training_accuracy,label="training accuracy")
# plt.plot(test_size,test_accuracy,label="test accuracy")
# plt.ylabel("Accuracy")
# plt.xlabel("test_size")
# plt.legend()

# plt.show()

In [ ]:
# appropriate test size find 
plt.figure()
training_accuracy = []
test_accuracy = []
# try n_neighbors from 1 to 10
test_size= numpy.arange(0.1, 0.9,0.05)
knn = KNeighborsClassifier(n_neighbors = 5)
for i in test_size:
    
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = i)
        knn.fit(X_train, y_train)
        training_accuracy.append(knn.score(X_train,y_train))
        test_accuracy.append(knn.score(X_test,y_test))
  

    
plt.plot(test_size,training_accuracy,label="training accuracy")
plt.plot(test_size,test_accuracy,label="test accuracy")
plt.ylabel("Accuracy")
plt.xlabel("test_size")
plt.legend()

plt.show()

In [ ]:
#split data into test data and train data
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.4)

neigh = KNeighborsClassifier(n_neighbors=5)


knn=neigh.fit(X_train,y_train)

knn.score(X_test,y_test)

In [ ]:
y_pred=knn.predict(X_test)

In [ ]:
#confusion matrix 
cf_matrix_n=confusion_matrix(y_test, y_pred)
cf_matrix_n

In [ ]:
#classification report
print(classification_report(y_test,y_pred))

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds=roc_curve(y_test, y_pred)
AUC=round(metrics.auc(fpr, tpr),2)

plt.plot([0,1], [0,1], linestyle='--', label='No Skill')
plt.plot(fpr, tpr, marker='.', label=f'knn AUC={AUC}')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
# show the plot
plt.show()


<b> Conclusion:</b>

In [ ]:
'''feature selection method affect our model,AUC decrease as compare to the first one'''

<b><h1> Logistics Regression</h1> </b>


In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Here we are using  both method for feature selection  means we takes union both feature union

In [ ]:
f_feature=list(set(basic_f0).union(set(basic_f1))) #final feature  just name

In [ ]:
f_feature

In [ ]:
# Now we need to use some domain knowlegde for selecting feature 
'''Group 1: {'Ease and convenient','Easy Payment option'}
   Group 2 :{'Order Time','Time saving','Delay of delivery person picking up food','Maximum wait time'}'''
f_feature=['Ease and convenient','Self Cooking','Maximum wait time','More restaurant choices',
 'Unaffordable','More Offers and Discount','Good Food quality','Good Tracking system','Age','Bad past experience']

In [ ]:
#select X,y 
X=fd_dlvry[f_feature]
y=fd_dlvry['Output']

In [ ]:
f_feature

In [ ]:
Lg=LogisticRegression()

In [ ]:
#
plt.figure()
training_accuracy = []
test_accuracy = []
test_size= numpy.arange(0.1, 0.9,0.05)

for i in test_size:
        t_s=[]
        tn_=[]
        for j in range(0,50):
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = i)
            Lg.fit(X_train, y_train)
            
            t_s.append(Lg.score(X_test,y_test))
            tn_.append(Lg.score(X_train,y_train))
        training_accuracy.append(np.mean(t_s))
        test_accuracy.append(np.mean(tn_))


    
plt.plot(test_size,training_accuracy,label="training accuracy")
plt.plot(test_size,test_accuracy,label="test accuracy")
plt.ylabel("Accuracy")
plt.xlabel("test_size")
plt.legend()

plt.show()




In [ ]:
#data split into train and test size 
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
Lg.fit(X_train,y_train)
Lg.score(X_test,y_test)

In [ ]:
y_pred=Lg.predict(X_test)

In [ ]:
cf_matrix=confusion_matrix(y_test, y_pred)
cf_matrix

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
thresolds = [0.2,0.3,0.4,0.5,0.55,0.6,0.62,0.65,0.7,0.8]
plt.figure(figsize=(10,6))
for i in thresolds:
    y_pred_proba = (Lg.predict_proba(X_test)[:,1]>=i).astype(int)
    fpr, tpr, m= metrics.roc_curve(y_test,  y_pred_proba)
    auc = metrics.roc_auc_score(y_test, y_pred_proba)
    plt.plot(fpr,tpr,label=f"data 1, auc={auc} thresold={i}")
    plt.legend(loc=4)



plt.xlabel('fpr')
plt.ylabel('tpr')
plt.show()

In [ ]:
'''In Knn weighted avg f1-score is good and AUC is great but problems in Knn is not learning something
knn just use more memory space for perform operation and mostly features are categorical so noise affect
on this algorithms. Here we can see if we set thresold 0.65 in Logistics regression we get better model  and here algorithms 
something learn 
'''

<b> Final Model </b>

In [ ]:
#select X,y 
X=fd_dlvry[f_feature]
y=fd_dlvry['Output']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.35)
Lg=LogisticRegression()
Lg.fit(X_train, y_train)
print("Training score=",Lg.score(X_train,y_train))
print("Test score=",Lg.score(X_test,y_test))

In [ ]:
y_pred_proba = (Lg.predict_proba(X_test)[:,1]>=0.65).astype(int)

In [ ]:
cf_matrix=confusion_matrix(y_test, y_pred_proba)
cf_matrix

In [ ]:
print(classification_report(y_test,y_pred_proba))

<b> thank you :) </b> 